|       | Description |
| ----------- | ----------- |
| Author      | Kamin Atsavasirilert (MSFE student at UIUC)|
| Date   | 02/08/2024        |
|Email|kamina2@illinois.edu|
|Idea| We price both American and European options using the fitted implied volatility from SSVI |

In [2]:
import QuantLib as ql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os
import regex as re
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy import interpolate
from scipy.optimize import minimize
import seaborn as sns
from tqdm import tqdm
from datetime import datetime, timedelta

In [3]:
def compute_phi(params, t, phi_type):
    global params_theta_t
    rho, eta, gamma = params
    theta_t = compute_theta_t(params_theta_t, t)
    if phi_type == "POWER-LAW1":
        return eta / theta_t ** (gamma)
    elif phi_type == "POWER-LAW2":
        return eta / (theta_t ** (gamma) * (1 + theta_t) ** (1-gamma)) 
    elif phi_type == "SQRT":
        return eta / (theta_t ** (0.5))
    elif phi_type == "HESTON":
        return (1 - (1 - np.exp(-gamma * theta_t) / (gamma * theta_t) )) / (gamma * theta_t)
    else:
        raise

def compute_theta_t(params, t):
    global kappa1, kappa2
    V, V_prime, theta = params
    theta_t = theta * t + (V - theta) * (1-np.exp(-kappa1 * t)) / kappa1 + (V_prime - theta) * kappa1 * ((1-np.exp(-kappa2 * t)) / kappa2 - (1-np.exp(-kappa1 * t)) / kappa1 ) / (kappa1 - kappa2)
    return theta_t

def compute_ssvi_inference(params_w_t, params_theta_t, t, k, phi_type):
    rho, eta, gamma = params_w_t
    theta_t = compute_theta_t(params_theta_t, t)
    phi = compute_phi(params_w_t, t, phi_type)
    return 0.5 * theta_t * ( 1 + rho * phi * k + np.sqrt((phi * k + rho)**2 + 1 - rho **2 ))

In [7]:
data_path = "../CQF_Optimal_Delta_Hedging_for_Options/data/put_call_2011_to_2023_no_dropna_with_div.csv"

tmp = pd.read_csv(data_path, parse_dates=['QUOTE_DATE', 'EXPIRE_DATE'])
tmp

,QUOTE_DATE,EXPIRE_DATE,DTE,UNDERLYING_LAST,STRIKE,P_BID,P_ASK,P_DELTA,P_GAMMA,P_VEGA,...,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,P_price,C_price,rf,Log-Strike,div
0,2011-01-04,2011-01-07,3.00,1270.37,1050.0,0.00,0.05,-0.00146,0.00000,0.00498,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,219.450,0.001200,-0.190528,0.0183
1,2011-01-04,2011-01-07,3.00,1270.37,1075.0,0.00,0.05,-0.00157,0.00006,0.00550,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,194.450,0.001200,-0.166997,0.0183
2,2011-01-04,2011-01-07,3.00,1270.37,1100.0,0.00,0.05,-0.00114,0.00010,0.00635,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,169.450,0.001200,-0.144008,0.0183
3,2011-01-04,2011-01-07,3.00,1270.37,1125.0,0.00,0.05,-0.00119,0.00015,0.00654,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,144.455,0.001200,-0.121535,0.0183
4,2011-01-04,2011-01-07,3.00,1270.37,1150.0,0.05,0.15,-0.00626,0.00035,0.02337,...,0.00000,0.00000,0.00000,0.00000,NaN,0.100,119.450,0.001200,-0.099556,0.0183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14704895,2023-09-29,2028-12-15,1904.04,4286.60,8800.0,2880.70,3072.60,-1.00000,0.00000,0.00000,...,0.00009,10.87378,-0.05829,11.09132,0.13139,2976.650,27.350,0.045752,0.480588,0.0156
14704896,2023-09-29,2028-12-15,1904.04,4286.60,9200.0,3186.30,3378.20,-1.00000,0.00000,0.00000,...,0.00007,9.24711,-0.04949,9.04105,0.13457,3282.250,22.050,0.045752,0.525040,0.0156
14704897,2023-09-29,2028-12-15,1904.04,4286.60,9600.0,3494.70,3686.60,-1.00000,0.00000,0.00000,...,0.00006,8.12839,-0.04411,7.69404,0.13909,3590.650,18.850,0.045752,0.567600,0.0156
14704898,2023-09-29,2028-12-15,1904.04,4286.60,10000.0,3805.00,3996.90,-1.00000,0.00000,0.00000,...,0.00000,6.16192,-0.03236,5.52154,0.13708,3900.950,12.700,0.045752,0.608422,0.0156


In [8]:
phi_type = "POWER-LAW1"
kappa1 = 5.5
kappa2 = 0.1
params_theta_t = np.array([3.020e-02,  3.727e-02,  4.138e-02])
params_w_t = np.array([-6.354e-01,  1.272e+00,  4.577e-01])

calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count = ql.Actual365Fixed()

In [11]:
DATE = '2023-01-20'
min_dte = 30
max_dte = 365*2
tmp_iv = tmp[(tmp['QUOTE_DATE'] == DATE)&(tmp['DTE'] >= min_dte)&(tmp['DTE'] <= max_dte)].copy()
tmp_iv

,QUOTE_DATE,EXPIRE_DATE,DTE,UNDERLYING_LAST,STRIKE,P_BID,P_ASK,P_DELTA,P_GAMMA,P_VEGA,...,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,P_price,C_price,rf,Log-Strike,div
13337982,2023-01-20,2023-02-24,35.0,3971.98,1000.0,0.00,0.05,0.00000,0.0,0.00375,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,2969.300,0.046817,-1.383754,0.0171
13337983,2023-01-20,2023-02-24,35.0,3971.98,1200.0,0.00,0.05,-0.00052,0.0,0.00357,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,2770.150,0.046817,-1.201432,0.0171
13337984,2023-01-20,2023-02-24,35.0,3971.98,1400.0,0.00,0.05,0.00000,0.0,0.00463,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,2571.000,0.046817,-1.047282,0.0171
13337985,2023-01-20,2023-02-24,35.0,3971.98,1600.0,0.00,0.10,-0.00025,0.0,0.00872,...,0.00000,5.85204,0.00000,0.00000,NaN,0.050,2371.850,0.046817,-0.913750,0.0171
13337986,2023-01-20,2023-02-24,35.0,3971.98,1800.0,0.05,0.15,-0.00079,0.0,0.01707,...,0.00000,15.12352,0.00000,0.00000,-0.00005,0.100,2172.750,0.046817,-0.795967,0.0171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13342019,2023-01-20,2024-12-20,700.0,3971.98,8400.0,3862.40,3900.70,-1.00000,0.0,0.00000,...,0.00000,0.68855,-0.01007,0.29815,0.18024,3881.550,1.200,0.041721,0.668953,0.0171
13342020,2023-01-20,2024-12-20,700.0,3971.98,8600.0,4045.60,4086.10,-1.00000,0.0,0.00000,...,-0.00002,0.62120,-0.00873,0.26537,0.18322,4065.850,1.075,0.041721,0.692484,0.0171
13342021,2023-01-20,2024-12-20,700.0,3971.98,8800.0,4228.70,4271.60,-1.00000,0.0,0.00000,...,-0.00001,0.54264,-0.00815,0.22753,0.18579,4250.150,0.925,0.041721,0.715473,0.0171
13342022,2023-01-20,2024-12-20,700.0,3971.98,9000.0,4411.80,4457.20,-1.00000,0.0,0.00000,...,0.00000,0.48728,-0.00700,0.20229,0.18842,4434.500,0.825,0.041721,0.737946,0.0171


# Pricing with Finite difference engine

In [ ]:
log_strike = 
dte = 
r = 
q = 
s0 = 
strike = 
actual_call = 
actual_put = 
exercise_date = 
today = 

am_exercise = ql.AmericanExercise(earliestDate=today, latestDate=exercise_date)

call_amer = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), am_exercise)
put_amer = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Put, strike), am_exercise)



call = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), 
                           ql.EuropeanExercise(exercise_date))
put = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Put, strike), 
                           ql.EuropeanExercise(exercise_date))


implied_vol = np.sqrt(compute_ssvi_inference(params_w_t, params_theta_t, dte/365, log_strike, phi_type=phi_type) / (dte/ 365))
riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(today, r, ql.Actual365Fixed()))
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(today, q, ql.Actual365Fixed()))
volatility = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), implied_vol, day_count))
initialValue = ql.QuoteHandle(ql.SimpleQuote(s0))
process = ql.BlackScholesMertonProcess(initialValue, dividendTS, riskFreeTS, volatility)

tGrid, xGrid = 2000, 2000
engine = ql.FdBlackScholesVanillaEngine(process, tGrid, xGrid)

call.setPricingEngine(engine)
put.setPricingEngine(engine)

call_amer.setPricingEngine(engine)
put_amer.setPricingEngine(engine)


print("dte:", dte, "r:", r, "q:", q, "s0:", s0, "strike:", strike, "implied_vol:", implied_vol, "today:", today)
print("exer_date:", exercise_date)
print("mid_call:", actual_call, "model price:",call.NPV())
print("mid_put:", actual_put, "model price:",put.NPV())
print("model call amer price:",call_amer.NPV())
print("model put amer price:",put_amer.NPV())

# Pricing with Monte Carlo engine

In [ ]:
###################### SETTINGs #########################
steps = 2
numPaths = 100000
rng = 'lowdiscrepancy'
#########################################################

log_strike = 
dte = 
r = 
q = 
s0 = 
strike = 
actual_call = 
actual_put = 
exercise_date = 
today = 

am_exercise = ql.AmericanExercise(earliestDate=today, latestDate=exercise_date)

call_amer = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), am_exercise)
put_amer = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Put, strike), am_exercise)



call = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), 
                           ql.EuropeanExercise(exercise_date))
put = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Put, strike), 
                           ql.EuropeanExercise(exercise_date))


implied_vol = np.sqrt(compute_ssvi_inference(params_w_t, params_theta_t, dte/365, log_strike, phi_type=phi_type) / (dte/ 365))
riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(today, r, ql.Actual365Fixed()))
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(today, q, ql.Actual365Fixed()))
volatility = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), implied_vol, day_count))
initialValue = ql.QuoteHandle(ql.SimpleQuote(s0))
process = ql.BlackScholesMertonProcess(initialValue, dividendTS, riskFreeTS, volatility)


eur_engine = ql.MCEuropeanEngine(process, rng, steps, requiredSamples=numPaths)
amer_engine = ql.MCAmericanEngine(process, rng, steps, requiredSamples=numPaths)

call.setPricingEngine(eur_engine)
put.setPricingEngine(eur_engine)

call_amer.setPricingEngine(amer_engine)
put_amer.setPricingEngine(amer_engine)


print("dte:", dte, "r:", r, "q:", q, "s0:", s0, "strike:", strike, "implied_vol:", implied_vol, "today:", today)
print("exer_date:", exercise_date)
print("mid_call:", actual_call, "model price:",call.NPV())
print("mid_put:", actual_put, "model price:",put.NPV())
print("model call amer price:",call_amer.NPV())
print("model put amer price:",put_amer.NPV())